In [1]:
def connect_db():
   # server = 
   # connection_string = 'mysql+mysqldb://root:pass@{}:3306/crawling?charset=utf8'.format(server)
    
    engine = create_engine(connection_string, pool_recycle = 3600, encoding='utf-8')
    Session = sessionmaker(bind=engine, expire_on_commit= True)

    session = Session()
    return session

In [2]:
# -*- coding: utf-8 -*-

import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, CHAR, Date, String, Time, Index, DateTime, TIMESTAMP, func
from sqlalchemy.dialects.mysql import INTEGER, BIT, VARCHAR, DATE, TINYINT, TIME, DOUBLE, TEXT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy import ForeignKeyConstraint
from sqlalchemy.orm import sessionmaker


Base = declarative_base()

class News(Base):
    __tablename__ = 'news'

    news_id        = Column(Integer, primary_key = True, autoincrement = True)
    title          = Column(VARCHAR(60))
    content        = Column(VARCHAR(60000))
    crawling_time  = Column(DateTime)

    def set_news_id(self,news_id):
        self.news_id = news_id
    def set_title(self,title):
        self.title = title
    def set_content(self,content):
        self.content = content
    def set_crawling_time(self,crawling_time):
        self.crawling_time = crawling_time    


In [3]:
def insert_into_db(class_name):
    
    session = connect_db()
    session.add(class_name)
    session.commit()
    
def update_into_db(class_name, filter_condition, update_dict):
    session = connect_db()
    
    rows = session.query(class_name).filter(filter_condition).update(update_dict)
    session.commit()

def select_from_db(class_name, filter_condition):
    
    session = connect_db()
    result = session.query(class_name).filter(filter_condition).all()

    
    return result

In [9]:
import requests
from bs4 import BeautifulSoup
import sys
import re

def get_naver_news_links():
    url = 'http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105'
    reg = r'^http://news.naver.com/main/read.nhn\?mode=LSD&mid=shm&sid1=105&oid=.+&aid=.+'
    
    soup = crawling_naver_news(url)
    
    div = soup.find('div', attrs = {'class': 'section_headline headline_subordi'})
    all_a = div.find_all('a', href= True)
    
    news_links = []
    for a in all_a:
        if re.findall(reg,a['href']):
            if a['href'] not in news_links:
                news_links.append(a['href'].encode('utf-8'))
    return news_links


def crawling_naver_news(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    return soup

def get_naver_title(soup):
    h3 = soup.find('h3', attrs = {'id':'articleTitle'})
    title = h3.get_text()
    
    return title

def get_naver_content(soup):
    
    body = soup.find('td', attrs = {'class' :'content'})

    div = body.find('div')

    [s.extract() for s in div('script')]
    
    div_list = div.select('#articleBodyContents')

    content = ""
    for div in div_list:
        content += div.get_text().strip()
        
    return content

 

In [5]:
#crawling

In [7]:
import datetime as dt
title_list = []
content_list = []
datetime_list = []
news_links = get_naver_news_links()
for url in news_links:
    print url
    soup = crawling_naver_news(url)
    title_list.append(get_naver_title(soup))
    content_list.append(get_naver_content(soup))
    datetime_list.append(dt.datetime.now())

print title_list
print content_list
print datetime_list

http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=081&aid=0002802652
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=138&aid=0002048403
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=079&aid=0002936683
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=277&aid=0003944377
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=421&aid=0002590225
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=008&aid=0003831437
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=030&aid=0002589138
http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=023&aid=0003259535
[u'\uc624\uac10 \uc778\uc2dd AI \ud50c\ub7ab\ud3fc\xb7\uace0\uc131\ub2a5 VR\u2026\u2018\ubaa8\ubc14\uc77c, \uadf8 \ub2e4\uc74c \uc694\uc18c\u2019 \uae38\uc744 \ucc3e\uc558\ub2e4', u'[MWC2017] MWC \ud3d0\ub9c9\u20263\ub300 \ud654\ub450, \u2018\uac24\ub7ed\uc2dcS8\xb75G\xb7IoT\u2019', u'\ubaa8\ubc14\uc77c \ub2e4\uc74c\uc

In [219]:
#db에 넣기

In [8]:
for i in range(len(title_list)):
    n1 = News()
    n1.set_title(title_list[i].encode('utf-8'))
    n1.set_content(content_list[i].encode('utf-8'))
    n1.set_crawling_time(datetime_list[i])
    
    insert_into_db(n1)
print 'done'

done
